<a href="https://colab.research.google.com/github/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/blob/main/Try%20CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try CatBoostRegressor

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# !pip install openpyxl -q
# import openpyxl

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# from datetime import datetime

# missing data
!pip install missingno -q
import missingno as msno

# for map
!pip install -q folium matplotlib
import folium
from matplotlib.colors import LinearSegmentedColormap, Normalize, to_hex

# for grid
!pip install geopy -q
from geopy.distance import geodesic

!pip install pandas numpy libpysal geopy esda -q
from geopy.distance import great_circle

# # for Moran
# !pip install pandas numpy libpysal geopy esda -q
# import libpysal
# from esda.moran import Moran

# linear regression
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import Lasso
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Set display options
%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.5 MB/s eta 0:00:00


In [3]:
apartments_initial = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/apartments4.xlsx', engine='openpyxl')

In [41]:
apartments = apartments_initial.copy()

In [5]:
import pickle
import requests
from io import BytesIO  # Import BytesIO

url_lat = 'https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/lat_bin.pkl'
url_lon = 'https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/lon_bin.pkl'

# Download lat_bin
response_lat = requests.get(url_lat)
lat_bin = pickle.load(BytesIO(response_lat.content))  # Load from the downloaded content

# Download lon_bin
response_lon = requests.get(url_lon)
lon_bin = pickle.load(BytesIO(response_lon.content))  # Load from the downloaded content

In [ ]:
# # удалить записи  если в ['Latitude_Bin', 'Longitude_Bin'] менее 3-х записей

# # Calculate the number of listings in each bin
# bin_counts = apartments.groupby(['Latitude_Bin', 'Longitude_Bin']).size().reset_index(name='count')

# # Filter out bins with less than 4 listings
# bins_to_keep = bin_counts[bin_counts['count'] >= 3][['Latitude_Bin', 'Longitude_Bin']]

# # Merge with the original DataFrame to keep only listings in the selected bins
# apartments1 = apartments.merge(bins_to_keep, on=['Latitude_Bin', 'Longitude_Bin'], how='inner')


In [6]:
apartments.columns

Index(['Listing ID', 'Host Since', 'Host Response Time', 'Host Response Rate',
       'Is Superhost', 'neighbourhood', 'Neighborhood Group', 'Postal Code',
       'Latitude', 'Longitude', 'Is Exact Location', 'Property Type',
       'Room Type', 'Accomodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Price',
       'Guests Included', 'Min Nights', 'Reviews', 'First Review',
       'Last Review', 'Overall Rating', 'Accuracy Rating',
       'Cleanliness Rating', 'Checkin Rating', 'Communication Rating',
       'Location Rating', 'Value Rating', 'Instant Bookable',
       'Host Since Years', 'Last Review Years', 'First Review Years',
       'Postal Code1', 'Location Rating Roughly', 'Reviews Log', 'Price Log',
       'Neighborhood Location Rating', 'Neighborhood Price', 'Latitude_Bin',
       'Longitude_Bin'],
      dtype='object')

In [42]:
apartments['Is Exact Location'] = apartments['Is Exact Location'].replace({'t': 1, 'f': 0}).fillna(0).astype(int)

In [36]:
apartments.dropna(subset=['Price'], inplace=True)

In [45]:
X = apartments.drop(['Listing ID', 'Host Since','Overall Rating','Postal Code', 'First Review',
       'Last Review', 'Reviews Log', 'Price Log','Price',
       'Neighborhood Location Rating', 'Neighborhood Price'], axis=1)  # Признаки
y = apartments['Price']  # Целевая переменная

In [46]:
!pip install catboost -q
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor, Pool  # Now CatBoost is available

num_features = X.select_dtypes(include='number').columns
cat_features = X.select_dtypes(exclude='number').columns

for col in cat_features:
    if X[col].dtype != 'str':
        X[col] = X[col].astype(str)

# Определяем признаки и таргет
X = X
y = y

# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Convert cat_features to a list
cat_features = cat_features.tolist()  # This is the key change

# Создаем модели для обучающей и тестовой выборок с указанием категориальных признаков
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# Инициализируем и обучаем модель
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(train_pool)

# Предсказания на тестовой выборке
y_pred = model.predict(test_pool)

# Вычисляем метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Выводим метрики
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

0:	learn: 35.8986624	total: 16.7ms	remaining: 16.7s
100:	learn: 23.0179759	total: 1.58s	remaining: 14.1s
200:	learn: 21.8907886	total: 3.03s	remaining: 12.1s
300:	learn: 21.1950515	total: 4.47s	remaining: 10.4s
400:	learn: 20.5727240	total: 5.86s	remaining: 8.75s
500:	learn: 20.0819952	total: 8.41s	remaining: 8.38s
600:	learn: 19.5722584	total: 10.9s	remaining: 7.26s
700:	learn: 19.1188676	total: 12.4s	remaining: 5.31s
800:	learn: 18.7367561	total: 14s	remaining: 3.47s
900:	learn: 18.3587535	total: 15.5s	remaining: 1.7s
999:	learn: 18.0292227	total: 17s	remaining: 0us
Mean Squared Error: 637.1124644033777
Mean Absolute Error: 16.22268598132093
R^2 Score: 0.5801647048499567
